# Algoritmo Minimax

O seguinte programa importa primeiramente a classe ConnectFour do módulo connect4 e de seguida define uma classe Minimax e uma subclasse Node. 

Neste programa estão definidos métodos para o funcionamento do Algoritmo Minimax com Alpha-Beta Pruning no jogo Connect 4. 

Este algoritmo de decisão é usado para que o computador possa determinar qual a próxima melhor jogada. Ele avalia todas as próximas jogadas possíveis (até uma certa profundidade) para conseguir fazer essa determinação.
Este algoritmo é depois otimizado com Alpha-Beta Pruning para reduzir o número de nós avaliados no espaço de busca, eliminando assim a necessidade de explorar caminhos que já se provaram inferiores a outros já examinados.

In [4]:
%run "./ConnectFour.ipynb"

In [9]:
from connect4 import ConnectFour

___Class Node___

A subclasse definida neste programa (class Node) é crucial para a criação da árvore de decisão usada pelo algoritmo Minimax. Ela também é usada para a representação do estado do jogo numa determinada altura, útil para avaliar as jogadas possíveis nesse ponto, verificar vitórias ou determinar se o jogo chegou ao fim.

Quanto à sua estrutura, este classe é muito simples. O seu método construturor __ __init__ __ tem 3 atributos 'board', 'parent', e 'move' que armazenam o estado atuyal do tabuleiro, o nó pai do nó atual e a jogada que levou ao estado atual do tabuleiro respetivamente.

O método __ __eq__ __ define a igualdade entre dois nós baseando-se nos estados do tabuleiro e o método __ __hash__ __ gera um hash baseado na representação do tabuleiro como uma string.

In [10]:
class Minimax:
    class Node:
        def __init__(self, board, parent, move):
            self.board = board
            self.parent = parent
            self.move = move

        def __eq__(self, other):
            return self.board == other.board 

        def __hash__(self):
            return hash(str(self.board))

O método __static_evaluation__ serve para a avaliação estática do tabuleiro. Ele verifica se algum dos jogadores venceu e retorna uma avaliação positiva (vitória do jogador X, _user_) ou negativa (vitória para o jogador O, _computador_). Caso constrário, retorna 0 (empate ou jogo não finalizado).

In [11]:
    def static_evaluation(self, board):
        winner = ConnectFour().win(board, ConnectFour().PLAYER_X)
        winner2 = ConnectFour().win(board, ConnectFour().PLAYER_O)
        if winner == True:
            return 100 
        elif winner2 == True:
            return -100
        else:
            return 0

O método __create_child__ serve para criar os filhos de um nó, representando possíveis jogadas seguintes. 
Para cada coluna válida, é simulada a jogada do jogador, cria um novo nó filho e adiciona-o à lista de filhos.

In [12]:
    def create_child(self, node, player):

        children = []
        for col in range(ConnectFour().COLUMNS):
            if ConnectFour().is_valid_column(node.board, col):
                row = ConnectFour().empty_row(node.board, col)
                new_board = ConnectFour().copy_board(node.board)
                ConnectFour().drop_token(new_board, row, col, player)
                new_node = self.Node(new_board, node, col)
                children.append(new_node)

        return children  

O método __minimax__ é o que implementa o algoritmo Minimax com Alpha-Beta Pruning.
Ele começa por verificar se a profundidade é zero, se algum dos jogadores venceu ou se o jogo está completo (empate) e se for case disso retorna a avaliação estática do tabuleiro e a jogada que levou a esse estado.

## Lógica para o jogador maximizador (maximizingPlayer = 'True')

- Se for a vez do jogador maximizador, o método procura a jogada que resulta no maior benefício (maximiza a avaliação). Ele itera sobre todos os possíveis movimentos ("filhos") do estado atual do jogo, chamando recursivamente o algoritmo Minimax para cada filho com a profundidade diminuída por um e alternando para o jogador minimizador.
- Se a avaliação de um filho exceder o valor atual de alpha, este é atualizado para esse valor. Se beta se tornar menor ou igual a alpha, a iteração é interrompida, já que os movimentos restantes não afetarão o resultado final.

## Lógica para o jogador minimizador (maximizingPlayer = 'False')

- Se for a vez do jogador minimizador, o método procura a jogada que resulta no menor benefício (minimiza a avaliação). Ele segue um processo semelhante ao do maximizador, mas procura minimizar a avaliação e atualiza a variável beta em vez de alpha.
- Se a variável beta se tornar menor ou igual a alpha a iteração é interrompida pelos movimentos restantes, já que esses movimentos não podem produzir um resultado melhor para o minimizador.

In [ ]:
def minimax(self, node, depth, alpha, beta, maximizingPlayer):
        if depth == 0 or ConnectFour().win(node.board, ConnectFour().PLAYER_X) or ConnectFour().win(node.board, ConnectFour().PLAYER_O) or ConnectFour().complete(): #game == 0 ou game over ou ja ganhou ou empate
            return Minimax().static_evaluation(node.board), node.move

        if maximizingPlayer:
            maxEval = float('-inf') 
            bestPlay = None
            for child in Minimax().create_child(node, ConnectFour().PLAYER_X):
                eval, _ = Minimax().minimax(child, depth - 1, alpha, beta, False)
                if eval > maxEval:
                    maxEval = eval
                    bestPlay = child.move
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return maxEval, bestPlay
        else:
            minEval = float('inf') 
            bestPlay = None
            for child in Minimax().create_child(node, ConnectFour().PLAYER_O):
                eval, _ = Minimax().minimax(child, depth - 1, alpha, beta, True)
                if eval < minEval:
                    minEval = eval
                    bestPlay = child.move
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return minEval, bestPlay